In [0]:
import numpy as np
from PIL import Image
import os
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.models import Sequential
from keras.utils import np_utils

## List Hardware

In [8]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 7572813978120951194, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11285974221
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 12254525737408764181
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"]

## Import Dataset

In [9]:
!wget https://pjreddie.com/media/files/mnist_train.csv

--2018-07-07 11:37:28--  https://pjreddie.com/media/files/mnist_train.csv
Resolving pjreddie.com (pjreddie.com)... 128.208.3.39
Connecting to pjreddie.com (pjreddie.com)|128.208.3.39|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 109575994 (104M) [application/octet-stream]
Saving to: ‘mnist_train.csv’

mnist_train.csv     100%[===================>] 104.50M   105MB/s    in 1.0s    

2018-07-07 11:37:30 (105 MB/s) - ‘mnist_train.csv’ saved [109575994/109575994]



In [10]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [11]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
!mkdir -p drive
!google-drive-ocamlfuse drive

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!unzip ./drive/data_set.zip

In [15]:
!ls data_set

cats  dogs  horses  Humans


In [0]:
dict_labels = {
    'cats': 0,
    'dogs': 1,
    'horses': 2,
    'Humans': 3
}
path = './data_set/'

In [42]:
x_ , y_ = [], []
classes = os.listdir( path )
for img_folder in classes:
  img_path = path + img_folder
  images = os.listdir( img_path )
  for img in images:
    x = Image.open( img_path + '/' + img )
    x = x.resize( (224,224) )
     
    x = np.array( x, dtype = np.float32 )
    #x = x/255.0
    
    x_.append( x )
    y_.append( dict_labels[img_folder] )

X = np.array(x_)
Y = np.array(y_)
print X.shape, Y.shape

(808, 224, 224, 3) (808,)


In [0]:
X = preprocess_input(X)

In [44]:
Y = np_utils.to_categorical(Y)
print Y.shape

(808, 4)


In [0]:
X, Y = shuffle( X, Y )

In [46]:
X_train, X_test, Y_train, Y_test = train_test_split( X, Y, test_size = 0.2 )
print X_train.shape, Y_train.shape

(646, 224, 224, 3) (646, 4)


In [0]:
resnet_model = ResNet50( input_shape = (224,224,3) , include_top = False, weights = 'imagenet' )

In [0]:
for ix in resnet_model.layers:
  ix.trainable = False

In [49]:
fc_layers = Sequential()
fc_layers.add( Flatten( input_shape = resnet_model.layers[-1].output_shape[1:] ) )
fc_layers.add( Dense(128, activation = 'relu') )
fc_layers.add( Dropout(0.4) )
fc_layers.add( Dense(4, activation = 'softmax') )
fc_layers.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               262272    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 4)                 516       
Total params: 262,788
Trainable params: 262,788
Non-trainable params: 0
_________________________________________________________________


In [50]:
final_model = Sequential( [ resnet_model, fc_layers ] )
final_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 1, 1, 2048)        23587712  
_________________________________________________________________
sequential_4 (Sequential)    (None, 4)                 262788    
Total params: 23,850,500
Trainable params: 262,788
Non-trainable params: 23,587,712
_________________________________________________________________


In [0]:
final_model.compile( optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'] )

In [52]:
final_model.fit( X_train, Y_train, batch_size = 16, epochs = 5, validation_data = [X_test, Y_test], shuffle = True) 

Train on 646 samples, validate on 162 samples
Epoch 1/5
646/646 [==============================] - 12s 19ms/step - loss: 0.4674 - acc: 0.8375 - val_loss: 0.1268 - val_acc: 0.9630
Epoch 2/5
646/646 [==============================] - 8s 12ms/step - loss: 0.0922 - acc: 0.9675 - val_loss: 0.1317 - val_acc: 0.9568
Epoch 3/5
646/646 [==============================] - 8s 12ms/step - loss: 0.1179 - acc: 0.9659 - val_loss: 0.1663 - val_acc: 0.9444
Epoch 4/5
646/646 [==============================] - 8s 12ms/step - loss: 0.0930 - acc: 0.9737 - val_loss: 0.1416 - val_acc: 0.9815
Epoch 5/5
646/646 [==============================] - 8s 12ms/step - loss: 0.0221 - acc: 0.9954 - val_loss: 0.1244 - val_acc: 0.9815


In [0]:
final_model.save('model.h5')

In [54]:
!ls

datalab  data_set  drive  mnist_train.csv  model.h5


In [0]:
from google.colab import files
files.download('model.h5')